## [Introduction to Cython for Solving Differential Equations](http://hplgit.github.io/teamods/cyode/cyode-sphinx/main_cyode.html)

The original post got from Hans Petter Langtangen (The above link).
I make it interactive and make it simpler, keeping the main points.
[Abolfazl Ziaeemhr]()

The first example involves a simple numerical method for solving a scalar first-order ordinary differential equation (ODE).

In [1]:
import timeit
import time

In [2]:
N = 2e6

### The initial pure Python code

In [3]:
from math import exp

def solver(f, I, dt, T, method):
    """
    Solve scalar ODE: 
    u'(t) = f(u,t), u(0)=I, 0 < t <= T
    method: numerical method to advance u one time step.
    dt: time step length.
    """
    N = int(round(float(T)/dt))
    u = I
    t = 0
    for n in range(N):  # may get memory error for large N
        u = method(u, t, f, dt)
        t += dt
    return u, t


def RK2(u, t, f, dt):
    """
    2nd-orderRunge-Kutta method for advancing u at time t
    to a new value at time t+dt. f is the right-hand side
    function f(u,t) in the equation u'=f.
    """
    K1 = dt*f(u, t)
    K2 = dt*f(u + 0.5*K1, t + 0.5*dt)
    unew = u + K2
    return unew

def problem1(u, t):
    return -u + 1

def problem2(u, t):
    """Right-hand side function f(u,t)."""
    return (- u + exp(-2*t))


In [4]:
def run(N, problem):
        I = 0
        T = 5
        dt = float(T)/(N-1)
        u, t = solver(problem, I, dt, T, RK2)
#         print (u, t)

In [5]:
# pure python
%timeit run(N, problem1)
%timeit run(N, problem2)

587 ms ± 61 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
868 ms ± 125 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%load_ext Cython

### only compile with cython

In [7]:
%%cython
def solver(f, I, dt, T, method):
    N = int(round(float(T)/dt))
    u = I  # previous time step (initial condition)
    t = 0
    for n in range(N):
        u = method(u, t, f, dt)
        t += dt
    return u, t

def RK2(u, t, f, dt):
    K1 = dt*f(u, t)
    K2 = dt*f(u + 0.5*K1, t + 0.5*dt)
    unew = u + K2
    return unew
    
from math import exp

def problem1(u, t):
    return -u + 1

def problem2(u, t):
    return - u + exp(-2*t)

In [8]:
%timeit run(N, problem1)
%timeit run(N, problem2)

441 ms ± 166 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
571 ms ± 168 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Declaring variables with types

In [9]:
%%cython
cpdef solver(f, double I, double dt, double T, method):
    cdef int N = int(round(float(T)/dt))
    cdef double u = I  # previous time step
    cdef double t = 0
    cdef int n
    for n in range(N):
        u = method(u, t, f, dt)
        t += dt
    return u, t

cpdef double RK2(double u, double t, f, double dt) except -10001:
    cdef double K1, K2, unew
    K1 = dt*f(u, t)
    K2 = dt*f(u + 0.5*K1, t + 0.5*dt)
    unew = u + K2
    return unew

from math import exp

cpdef double problem1(double u, double t) except -10001:
    return -u + 1

cpdef double problem2(double u, double t) except -10001:
    return -u + exp(-2 * t)


In [10]:
%timeit run(N, problem1)
%timeit run(N, problem2)

326 ms ± 103 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
640 ms ± 215 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


replacing the calls to f and method by the actual function names problem (in RK2) and RK2 (in solver)

In [11]:
%%cython
cpdef solver(f, double I, double dt, double T, method):
    cdef int N = int(round(float(T)/dt))
    cdef double u = I  # previous time step
    cdef double t = 0
    cdef int n
    for n in xrange(N):
        #u = method(u, t, f, dt)
        u = RK2(u, t, f, dt)
        t += dt
    return u, t

cpdef double RK2(double u, double t, f, double dt):
    cdef double K1, K2, unew
    #K1 = dt*f(u, t)
    #K2 = dt*f(u + 0.5*K1, t + 0.5*dt)
    K1 = dt*problem1(u, t)
    K2 = dt*problem1(u + 0.5*K1, t + 0.5*dt)
    unew = u + K2
    return unew
    
cpdef double problem1(double u, double t):
    return -u + 1

from math import exp

cpdef double problem2(double u, double t):
    return - u + exp(-2*t)



In [12]:
%timeit run(N, problem1)
%timeit run(N, problem2)

14.4 ms ± 575 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
14.7 ms ± 321 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Inspecting what Cython has done

In [13]:
%%cython --annotate
cpdef solver(f, double I, double dt, double T, method):
    cdef int N = int(round(float(T)/dt))
    cdef double u = I  # previous time step
    cdef double t = 0
    cdef int n
    for n in range(N):
        u = method(u, t, f, dt)
        t += dt
    return u, t

cpdef double RK2(double u, double t, f, double dt) except -10001:
    cdef double K1, K2, unew
    K1 = dt*f(u, t)
    K2 = dt*f(u + 0.5*K1, t + 0.5*dt)
    unew = u + K2
    return unew

from math import exp


cpdef double problem1(double u, double t) except -10001:
    return -u + 1


cpdef double problem2(double u, double t) except -10001:
    return -u + exp(-2 * t)


### Proper treatment of functions as arguments to functions

In [14]:
%%cython
cdef class Problem:
    cpdef double rhs(self, double u, double t) except -10001:
        return 0

from math import exp

cdef class Problem1(Problem):
    cpdef double rhs(self, double u, double t) except -10001:
        return - u + 1

cdef class Problem2(Problem):
    cpdef double rhs(self, double u, double t) except -10001:
        return - u + exp(-2*t)

cdef class ODEMethod:
    cpdef double advance(self, double u, double t, Problem p,
                        double dt) except -10001:
        return 0

cdef class Method_RK2(ODEMethod):
    cpdef double advance(self, double u, double t, Problem p,
                         double dt) except -10001:
        cdef double K1, K2, unew
        K1 = dt*p.rhs(u, t)
        K2 = dt*p.rhs(u + 0.5*K1, t + 0.5*dt)
        unew = u + K2
        return unew

# Create names compatible with ode0.py
RK2 = Method_RK2()
problem1 = Problem1()
problem2 = Problem2()


cpdef solver(Problem f, double I, double dt,
             double T, ODEMethod method):
    cdef int N = int(round(float(T)/dt))
    cdef double u = I  # previous time step
    cdef double t = 0
    cdef int n
    for n in xrange(N):
        u = method.advance(u, t, f, dt)
        t += dt
    return u, t

In [15]:
%timeit run(N, problem1)
%timeit run(N, problem2)

21.9 ms ± 174 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
368 ms ± 69.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### remove the except * constructions

In [16]:
%%cython
cdef class Problem:
    cpdef double rhs(self, double u, double t):
        return 0

from math import exp

cdef class Problem1(Problem):
    cpdef double rhs(self, double u, double t):
        return - u + 1

cdef class Problem2(Problem):
    cpdef double rhs(self, double u, double t):
        return - u + exp(-2*t)

cdef class ODEMethod:
    cpdef double advance(self, double u, double t, Problem p,
                        double dt):
        return 0

cdef class Method_RK2(ODEMethod):
    cpdef double advance(self, double u, double t, Problem p,
                         double dt):
        cdef double K1, K2, unew
        K1 = dt*p.rhs(u, t)
        K2 = dt*p.rhs(u + 0.5*K1, t + 0.5*dt)
        unew = u + K2
        return unew

# Create names compatible with ode0.py
RK2 = Method_RK2()
problem1 = Problem1()
problem2 = Problem2()


cpdef solver(Problem f, double I, double dt,
             double T, ODEMethod method):
    cdef int N = int(round(float(T)/dt))
    cdef double u = I  # previous time step
    cdef double t = 0
    cdef int n
    for n in xrange(N):
        u = method.advance(u, t, f, dt)
        t += dt
    return u, t

In [17]:
%timeit run(N, problem1)
%timeit run(N, problem2)

22 ms ± 2.96 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
279 ms ± 88.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
# handling mathematical functions

In [19]:
%%cython
cdef class Problem:
    cpdef double rhs(self, double u, double t):
        return 0

# from math import exp
cdef extern from "math.h":
    double exp(double)

cdef class Problem2(Problem):
    cpdef double rhs(self, double u, double t):
        return - u + exp(-2*t)

cdef class ODEMethod:
    cpdef double advance(self, double u, double t, Problem p,
                        double dt):
        return 0

cdef class Method_RK2(ODEMethod):
    cpdef double advance(self, double u, double t, Problem p,
                         double dt):
        cdef double K1, K2, unew
        K1 = dt*p.rhs(u, t)
        K2 = dt*p.rhs(u + 0.5*K1, t + 0.5*dt)
        unew = u + K2
        return unew

# Create names compatible with ode0.py
RK2 = Method_RK2()
problem2 = Problem2()


cpdef solver(Problem f, double I, double dt,
             double T, ODEMethod method):
    cdef int N = int(round(float(T)/dt))
    cdef double u = I  # previous time step
    cdef double t = 0
    cdef int n
    for n in xrange(N):
        u = method.advance(u, t, f, dt)
        t += dt
    return u, t

In [20]:
%timeit run(N, problem2)

83.6 ms ± 26.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [21]:
# using exp from libc.math

In [22]:
%%cython
cdef class Problem:
    cpdef double rhs(self, double u, double t):
        return 0

# from math import exp
# cdef extern from "math.h":
#     double exp(double)
from libc.math cimport exp

cdef class Problem2(Problem):
    cpdef double rhs(self, double u, double t):
        return - u + exp(-2*t)

cdef class ODEMethod:
    cpdef double advance(self, double u, double t, Problem p,
                        double dt):
        return 0

cdef class Method_RK2(ODEMethod):
    cpdef double advance(self, double u, double t, Problem p,
                         double dt):
        cdef double K1, K2, unew
        K1 = dt*p.rhs(u, t)
        K2 = dt*p.rhs(u + 0.5*K1, t + 0.5*dt)
        unew = u + K2
        return unew

# Create names compatible with ode0.py
RK2 = Method_RK2()
problem2 = Problem2()


cpdef solver(Problem f, double I, double dt,
             double T, ODEMethod method):
    cdef int N = int(round(float(T)/dt))
    cdef double u = I  # previous time step
    cdef double t = 0
    cdef int n
    for n in xrange(N):
        u = method.advance(u, t, f, dt)
        t += dt
    return u, t

In [23]:
%timeit run(N, problem2)

85.8 ms ± 26.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [25]:
# use exp from numpy

In [26]:
%%cython
cdef class Problem:
    cpdef double rhs(self, double u, double t):
        return 0

# from math import exp
# cdef extern from "math.h":
#     double exp(double)
# from libc.math cimport exp
from numpy import exp

cdef class Problem2(Problem):
    cpdef double rhs(self, double u, double t):
        return - u + exp(-2*t)

cdef class ODEMethod:
    cpdef double advance(self, double u, double t, Problem p,
                        double dt):
        return 0

cdef class Method_RK2(ODEMethod):
    cpdef double advance(self, double u, double t, Problem p,
                         double dt):
        cdef double K1, K2, unew
        K1 = dt*p.rhs(u, t)
        K2 = dt*p.rhs(u + 0.5*K1, t + 0.5*dt)
        unew = u + K2
        return unew

# Create names compatible with ode0.py
RK2 = Method_RK2()
problem2 = Problem2()


cpdef solver(Problem f, double I, double dt,
             double T, ODEMethod method):
    cdef int N = int(round(float(T)/dt))
    cdef double u = I  # previous time step
    cdef double t = 0
    cdef int n
    for n in xrange(N):
        u = method.advance(u, t, f, dt)
        t += dt
    return u, t

In [27]:
%timeit run(N, problem2)

3.55 s ± 438 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
